<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/Llama3.1-8b/answer_Generation_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [5]:
!pip install datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens

In [6]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

#**loading the model**

In [3]:
# Load the model and tokenizer
model_id = "meta-llama/Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("من اسس السعودية الثالثة"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'من اسس السعودية الثالثة\nالمنطقة الإدارية التي يقع فيها بلدك\nالمنطقة الإدار'}]


# helper functions

In [ ]:
torch.cuda.empty_cache()

with the likelihood:)

In [7]:
def generate_answer(question, num_samples, model, tokenizer):
    prompt = f" أجب على السؤال التالي بجملة واحدة فقط موجزة ولكن كاملة باللغة العربية\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt
    prompt_tokens = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate responses
    outputs = model.generate(
        **prompt_tokens,
        max_length=150,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        num_return_sequences=num_samples,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # Extract generated sequences and scores
    sequences = outputs.sequences
    scores = outputs.scores

    # Process each generated sequence
    results = []
    for i, seq in enumerate(sequences):
        # Get the text
        decoded_text = tokenizer.decode(seq, skip_special_tokens=True)
        #clean the output like what they did
        answer = decoded_text.split("Answer:")[-1].strip()
        strings_to_filter_on = ['.', '\n', 'Q:', 'A:', 'question:', 'answer:', 'Question:', 'Answer:', 'Questions:', 'questions:', 'QUESTION:', 'ANSWER:']
        for string in strings_to_filter_on:
            if string in answer:
                answer = answer.split(string)[0].strip()

        # Calculate log probabilities (similar to reference code)
        # We'll use model.forward to get detailed probabilities
        with torch.no_grad():
            # Only consider tokens after the prompt
            prompt_length = len(prompt_tokens.input_ids[0])
            gen_tokens = seq[prompt_length-1:]  # Include one overlap token

            # Feed the sequence to the model
            target_ids = gen_tokens.clone()
            target_ids[0] = -100  # so it Dose't compute loss for the overlap token
            model_output = model(torch.reshape(gen_tokens, (1, -1)), labels=target_ids)

            # Get average negative log likelihood
            avg_neg_log_likelihood = model_output.loss

        results.append({
            'text': answer,  # This will now contain the cleaned answer
            'avg_neg_log_likelihood': avg_neg_log_likelihood.item(),
        })

    return results

# main function to to load questions and gerate answers and save them to a file

In [11]:
# Main function
def main():

    # Step 1: LOAD DATA
    file_path = '/content/Arabic-SQuAD.json'

    if os.path.exists(file_path):

        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:

      # Load the XQuAD dataset for Arabic (ar)
      # xquad_dataset = load_dataset("xquad", "xquad.ar")
      ds = load_dataset("Mostafa3zazi/Arabic_SQuAD")
      # Load the validation set (the only available split)
      test_set = ds['train']

     # Create an empty list to store the data
      data = []
      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['text'] #chenged !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
          # Append the data as a dictionary to the list
          data.append({
                  "question_id": i,
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })
    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)


  # Step 2: GENERATE ANSWERS
  #empty list to store the results
    results = []

    for index, row in df.sample(n=5).iterrows():
        question = row['question','Question']
        original_answer = row.get('answer','Answer' ,"")  # Use empty string if no original answer provided
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(question=question, num_samples=10, model=model, tokenizer=tokenizer)
        # Extract text answers and log probabilities
        generated_answers = [result['text'] for result in generated_results]
        avg_neg_log_likelihoods = [result['avg_neg_log_likelihood'] for result in generated_results]

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers,
            "avg_neg_log_likelihoods": avg_neg_log_likelihoods,
        })
    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)

    #SAVE GENERATION
    filename = "generated_answers_Llama3.1-LOG.json"

    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    files.download(filename)


# Run the main function
if __name__ == "__main__":
    main()

KeyError: 'key of type tuple not found and not a MultiIndex'

In [10]:
import json
import re
from tqdm import tqdm


tydiqa_train_path = '/content/tydiqa-goldp-dev-arabic.json'

with open(tydiqa_train_path, "r", encoding='utf-8') as reader:
    input_data = json.load(reader)["data"]

arabic_data = []
for entry in tqdm(input_data):
    for paragraph in entry["paragraphs"]:
        if 'arabic' in paragraph['qas'][0]['id']:
            arabic_data.append(
                {
                    'paragraphs' : [paragraph],
                    'title' : entry['title']
                }
            )

print(len(arabic_data))

arabic_data = {
    "data": arabic_data,
    "version": "TyDiQA-GoldP-1.1-for-SQuAD-1.1",
    'lang' : 'ar'
}

with open('/content/tydiqa-arabic.json', "w",encoding='utf-8') as writer:
    json.dump(arabic_data, writer)

100%|██████████| 921/921 [00:00<00:00, 1060668.31it/s]

921
